# Spatial analysis

After running PIPEX on our images, we would like to load the images and results into a `SpatialData` Object. Afterwards, we can interactively look at it with Napari  or run more spatial analyses with SquidPy

In [1]:
from anndata import read_h5ad
from pathlib import Path
from napari_spatialdata import Interactive
import numpy as np
from spatialdata import SpatialData
from spatialdata.models import Image2DModel, Labels2DModel, TableModel
import tifffile

## Data download

You could use the data generated during the course while running PIPEX. Or you might also download the data from [here](https://ell-vault.stanford.edu/dav/fredbn/www/I2K/pipex_data.zip) and unzip it into the data folder inside the `I2K2024-MTIWOKSHOP` folder.

If it didn't work, you can uncomment the lines in the cell below and run them.

In [ ]:
# from utils import download_and_unzip

# file_url = "https://ell-vault.stanford.edu/dav/fredbn/www/I2K/pipex_data.zip"
# output_folder = '../data'
# download_and_unzip(file_url, output_folder)

## SpatialData construction

We can begin by stating the directories of the pipex results, images and where we would like to store the SPatialData Objects as a Zarr.

In [34]:
pipex_dir = Path("../data/data/analysis")
images_dir = Path("../data/data")
sdata_file = Path("../data/pipex.zarr")

In [ ]:
channels = []
channel_imgs = []
for channel_img_file in sorted(images_dir.glob("*.tif")):
    channel_img = tifffile.imread(channel_img_file)
    channels.append(channel_img_file.stem)
    channel_imgs.append(channel_img)
img = np.stack(channel_imgs)
img.shape

In [ ]:
labels_file = pipex_dir / "segmentation_data.npy"
labels = np.load(labels_file).astype(np.uint16)
labels.shape

In [ ]:
cells_file = pipex_dir / "downstream" / "anndata_TissUUmaps.h5ad"
cells = read_h5ad(cells_file) 
cells

We need to add some information to relate the AnnData information in the table to the cells detected and shown in the labels.

In [8]:
cells.uns["spatialdata_attrs"] = {"region": ["labels"],
                                  "region_key": "sample_id",
                                  "instance_key": "id"}
cells.obs["sample_id"] = "labels"
cells.obs["sample_id"] = cells.obs["sample_id"].astype("category")

In [ ]:
sdata = SpatialData(
    images={"images": Image2DModel.parse(img, dims=("c", "y", "x"), c_coords=channels)},
    labels={"labels": Labels2DModel.parse(labels, dims=("y", "x"))},
    tables={"table": TableModel.parse(cells)},
)
sdata.write(sdata_file, overwrite=True)
sdata

In [ ]:
Interactive(sdata)

## Spatial single-cell analysis

We can begin by reading the spatial data file constructed on the previous subsection.

In [ ]:
sdata_file = "../data/pipex.zarr"
sdata = SpatialData.read(sdata_file)
sdata

We can import squidpy and scanpy to perform analysis on our files, but most of these analyses have already been performed and are saved in the appended tables.

In [3]:
# import scanpy as sc
import squidpy as sq

In [ ]:
sdata["table"]

In [ ]:
sdata["table"].var_names, sdata["table"].obs_names

We can visualize the lolcation of every cluster in the plot with the following function.

In [ ]:
sq.pl.spatial_scatter(sdata["table"], shape=None, color="leiden", size=50)

We can compute a spatial graph by running `spatial_neighbours` function in the `gr` module (graph). And then we can run `spatial_scatter` once more from the `pl` (plot) module, adding the `connectivity_key` parameter.

*Note*: Feel free to play around with the radius parameter and see what happens when you increase it to 100.

In [ ]:
sq.gr.spatial_neighbors(sdata["table"], radius=30.0, coord_type="generic")
sq.pl.spatial_scatter(
    sdata["table"],
    color="leiden",
    connectivity_key="spatial_connectivities",
    edges_color="black",
    shape=None,
    edges_width=1,
    size=30,
)

We could as well rerun and plot (or only plot) the neighbourhood enrichment analysis.

In [ ]:
# sq.gr.spatial_neighbors(sdata["cells"])
# sq.gr.nhood_enrichment(sdata["cells"], cluster_key="leiden")
sq.pl.nhood_enrichment(sdata["table"], cluster_key="leiden", figsize=(5, 5))

We could overlay the graphs with the an image to see how it looks like.

In [56]:
spatial_key = "spatial"
library_id = "labels"
adata = sdata["table"]

adata.uns[spatial_key] = {library_id: {}}
adata.uns[spatial_key][library_id]["images"] = {}
adata.uns[spatial_key][library_id]["images"] = {"hires": sdata["images"].loc["DAPI"].to_numpy()}
adata.uns[spatial_key][library_id]["scalefactors"] = {
    "tissue_hires_scalef": 1,
    "spot_diameter_fullres": 0.5,
}

In [ ]:
adata.uns['spatial']

In [ ]:
sq.pl.spatial_scatter(adata, color="leiden")